In [1]:
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from trl import setup_chat_format

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_to_source_model = "C:\\Users\\USER_ELISEY\\qwen"
path_to_finetuned_model = "C:\\Users\\USER_ELISEY\\hakaton\\russia_mini_chad\\checkpoint-500"

# Compare models

In [3]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model
casual_model = AutoModelForCausalLM.from_pretrained(
    path_to_source_model,
    quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_source_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [4]:
casual_model, tokenizer = setup_chat_format(casual_model, tokenizer)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    path_to_finetuned_model,
    quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_finetuned_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [6]:
model, tokenizer = setup_chat_format(model, tokenizer)

## Get answers

In [7]:
def generate_answer(model, prompt):
    chat = [
        { "role": "user", "content": prompt },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

    return(tokenizer.decode(outputs[0]))

# Comprasion

In [18]:
q1 = "Who is Alexandr Nevskiy??"
q2 = "Как звали сына Сталина?"
q3 = "Кто такой Брежнев?"

In [19]:
print(generate_answer(model, q1))

<|im_start|>user
Who is Alexandr Nevskiy??<|im_end|>
<|im_start|>assistant
Александра Невская (, , ; 19 сентября 1901, Москва — 19 октября 1981, Москва) — русский писатель, публицист, журналист, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель, общественный деятель


In [10]:
print(generate_answer(model, q2))

<|im_start|>user
Как звали сына Сталина?<|im_end|>
<|im_start|>assistant
Сын Сталина — имя, которое было названо в 1936 году, когда Сталин был избран на должность президента СССР. В 1936 году Сталин был избран на должность президента СССР, но в 1938 году был избран на должность президента Советского Союза. В 1938 году Сталин был избран на должность президента Советского Союза, но в 1940 году был избран на должность президента СССР. В 1940 году Сталин был избран


In [11]:
print(generate_answer(model, q3))

<|im_start|>user
Кто такой Брежнев?<|im_end|>
<|im_start|>assistant
Брежнев (, , ; 19 мая 1890, Москва — 17 мая 1972, Москва) — российский политический деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный деятель, государственный


In [12]:
# # Free gpu memory
# import numba
# numba.cuda.close()

In [20]:
print(generate_answer(casual_model, q1))

<|im_start|>user
Who is Alexandr Nevskiy??<|im_end|>
<|im_start|>assistant
Based on the information provided in the text, here are some possible answers to the question "Who is Alexandr Nevskiy?":

1. Alexandr Nevskiy is a Russian mathematician and professor at the University of Michigan.
2. He is known for his contributions to the theory of algebraic curves and the theory of algebraic varieties.
3. He has published numerous papers and books on these topics, including "Algebraic Curves" (1981) and "Algebraic Varieties" (1988).
4. He has also been a professor at the University of Michigan since 1988.
5. He has received several awards and honors, including the Fields Medal in 200


In [14]:
print(generate_answer(casual_model, q2))

<|im_start|>user
Как звали сына Сталина?<|im_end|>
<|im_start|>assistant
Based on the information provided in the text, the name of the son of Stalin is not mentioned. The text only mentions that Stalin was the leader of the Soviet Union and that he was born in 1903. There is no mention of his name or any other information about his son.<|im_end|>


In [15]:
print(generate_answer(casual_model, q3))

<|im_start|>user
Кто такой Брежнев?<|im_end|>
<|im_start|>assistant
The person who is the subject of the question is the former Soviet leader and Prime Minister of the Russian Federation, Boris Yeltsin.<|im_end|>
